# Get Datasets

In [1]:
import pandas as pd
import numpy as np
from ucimlrepo import fetch_ucirepo
from openml import datasets

preprocessing_log = pd.DataFrame(columns=['did', 'Dataset Name', 'No. of Rows', 'No. of Columns', 'No. of Extracted Meta-Features', 'One-Hot Encoded?', 'Label Encoded?'])

openml_datasets_index = pd.read_csv('openml_datasets_index.csv')
uci_datasets_index = pd.read_csv('uci_datasets_index.csv')

openml_index = openml_datasets_index
uci_index = uci_datasets_index

In [ ]:
from math import ceil
openml_index = openml_datasets_index
uci_index = uci_datasets_index


openml_index = openml_index[openml_index['NumberOfClasses'] == 2]
openml_index = openml_index[openml_index['NumberOfMissingValues'] == 0]
openml_index = openml_index[openml_index['NumberOfInstancesWithMissingValues'] == 0]

passing_dids = []
for _row in openml_index.iterrows():
    row = _row[1]
    
    dataset_size = row['NumberOfInstances']
    pos_prop = row['MajorityClassSize'] / dataset_size
    neg_prop = row['MinorityClassSize'] / dataset_size

    sample_size = 100
    test_size = 0.5

    pos_number_sample_test = ceil(pos_prop * dataset_size * test_size)
    neg_number_sample_test = ceil(neg_prop * dataset_size * test_size)

    if pos_number_sample_test > sample_size and neg_number_sample_test > sample_size:
        passing_dids.append(row['did'])

openml_index = openml_index[openml_index['did'].isin(passing_dids)]

openml_index

# Run

In [ ]:
from run_tests import run_tests

did_list = openml_index['did'].tolist()
i = 0
for did in did_list:
    i+=1
    print(f"ITER. {i}")
    try:
        dataset = datasets.get_dataset(int(did))
    except:
        continue

    dataset_name = dataset.name
    X, y, categorical_indicator, attribute_names = dataset.get_data(
        dataset_format="dataframe", target=dataset.default_target_attribute
    )
    dataset_shape, num_meta_features_extracted, one_hot_encoded, label_encoded = run_tests(X, y, categorical_indicator)
    
    if dataset_shape[1] > 100:
        continue
    
    preprocessing_log.loc[len(preprocessing_log.index)] = [did, dataset_name, dataset_shape[0], dataset_shape[1], num_meta_features_extracted, one_hot_encoded, label_encoded]

preprocessing_log